In [3]:
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, SentimentOptions, EmotionOptions

In [4]:
authenticator = IAMAuthenticator('ZtAL7sIQ58MB8UrjOeYR_fvko36zVObg57OQ8y3FftDn')
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2019-07-12',
    authenticator=authenticator
)
natural_language_understanding.set_service_url('https://api.us-south.natural-language-understanding.watson.cloud.ibm.com/instances/90eb04a6-053b-4fdd-a09a-2005d9fe421b')

In [13]:
# open json
with open('./processed_bbc_test.json') as f:
  data = json.load(f)

# append analysis
updated_json = analyze_newssource(data)

# write json
with open('./processed_bbc_test_updated.json', 'w') as f:
  json.dump(updated_json, f)


+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50
+50

KeyboardInterrupt: 

In [71]:
print(newssource_analysis)

'sentiment': {'document': {'score': 0.376071, 'label': 'positive'}}, 'language': 'en', 'emotion': {'document': {'emotion': {'sadness': 0.592849, 'joy': 0.605757, 'fear': 0.519499, 'disgust': 0.041778, 'anger': 0.11257}}}}, '02/10/2020_Coronavirus: Brighton GP practice closes after staff member tests positive': {'usage': {'text_units': 1, 'text_characters': 5554, 'features': 2}, 'sentiment': {'document': {'score': -0.658057, 'label': 'negative'}}, 'language': 'en', 'emotion': {'document': {'emotion': {'sadness': 0.317763, 'joy': 0.164502, 'fear': 0.439561, 'disgust': 0.493306, 'anger': 0.121955}}}}, '02/10/2020_Coronavirus labs to begin testing in Scotland': {'usage': {'text_units': 1, 'text_characters': 3141, 'features': 2}, 'sentiment': {'document': {'score': 0.366546, 'label': 'positive'}}, 'language': 'en', 'emotion': {'document': {'emotion': {'sadness': 0.129294, 'joy': 0.657556, 'fear': 0.547609, 'disgust': 0.064129, 'anger': 0.091021}}}}, '02/10/2020_Nissan to shut Japan factory 

In [12]:
def analyze_newssource(json):
    no_articles = 0
    newssource_analysis = {}

    for date in json:
        for article in json[date]:
            no_articles += 1
            
            if(no_articles % 50 == 0):
                print('+50')

            json[date].append({'new_key' : 'new_value'})
            #analysis = send_analysis_request(text)

    print('done')
    return json

In [72]:
def send_analysis_request(text):
    response = natural_language_understanding.analyze(
        text=text,
        features=Features(sentiment=SentimentOptions(), emotion=EmotionOptions())
    ).get_result()

    return response